In [ ]:
# Imports
import pandas as pd
import numpy as np

from answers import (
    br_norm_answer,
    fa_norm_answer,
    pos_answer,
    break_pos_answer,
    br_trd_norm_answer,
    fa_trd_norm_answer,
    fa_p_answer,
    br_p_answer,
    pnl_cmp_answer,
    pnl_cmp_v2_answer
)

# Paths
DATA = "../data"
POS_DIR = f"{DATA}/positions"
TRD_DIR = f"{DATA}/trades"

pd.set_option("display.float_format", lambda v: f"{v:,.4f}")

## Step 1 — Positions Recon (as of 2025‑09‑11)

**Prompt:**
1) Load Fund Admin and Broker positions.

2) Normalize to common schema: `'date', 'security_description', 'security_id', 'quantity', 'price','multiplier', 'avg_cost', 'start_of_month_price'`.

3) Merge on `security_id` (prefer `validate='one_to_one'`).

4) Produce break table: `unique_id, qty_admin, qty_broker, qty_diff, avg_cost_admin, avg_cost_broker, avg_cost_diff`. 

In [ ]:
# View the data and describe the data verbally

In [ ]:
# Load positions
fa_raw = pd.read_csv(f"{POS_DIR}/fund_admin_positions_2025-09-11.csv", parse_dates=["date"])
br_raw = pd.read_csv(f"{POS_DIR}/broker_positions_2025-09-11.csv", parse_dates=["date"])

#### Get a feel for the data

In [ ]:
# View the data and describe the data verbally

#### Normalize

In [ ]:
# Helpers as per LTA schema requirements
# Converters to update broker/fund admin column names to LTA schema names
pos_col_converters = {
'date': 'date',
'ticker': 'security_description',
'unique_id': 'security_id',
'quantity': 'quantity',
'price': 'price',
'contract_multiplier': 'multiplier',
'average_cost': 'avg_cost'
}

# LTA only keeps these columns in our position schemas
keep_pos_columns = [
'date',
'security_description',
'security_id',
'quantity',
'price',
'multiplier',
'avg_cost',
'start_of_month_price',
]

In [ ]:
# I have normalized the fund admin data for you
# Can you walk me through what the code is doing line by line?

In [ ]:
def normalize_fa_pos(fa_pos_raw, lot_col, avg_cost_col):
    # 1. ?
    d = fa_pos_raw.copy()

    # 2. ?
    d['lot_signed'] = d[lot_col] * d['lot_sign']

    # 3. ?
    d['qty_times_price'] = d[avg_cost_col] * d['lot_signed'] 
    
    pos_group_cols = [
        'date',
        'security_description',
        'price',
        'contract_multiplier',
        'start_of_month_price',
        'unique_id',
        ]
    
    agg = d.groupby(pos_group_cols).agg(
        quantity=('lot_signed', 'sum'), total_qty_times_price=('qty_times_price', 'sum')
        )
    
    agg['avg_cost'] = agg['total_qty_times_price'] / agg['quantity']
    
    # 4. ?
    agg = agg.reset_index()
    # 5. ?
    agg = agg.rename(columns=pos_col_converters)
    # 6. ?
    agg = agg[keep_pos_columns]
    
    return agg

In [ ]:
display(fa_raw)
fa_norm = normalize_fa_pos(fa_raw, 'lot_qty', 'average_cost').sort_values(by='security_id')
display(fa_norm)

In [ ]:
# TODO: normalize broker data to LTA Schema Requirements, sort values by security_id
# Name the normalized dataframe br_norm
# Note that this is FAR easier than the normalization above -- you do not need a function

In [ ]:
br_norm = pd.DataFrame() # UPDATE CODE HERE

In [ ]:
if br_norm.equals(br_norm_answer):
    print('Congratulations')
else:
    print('Your answer: ')
    display(br_norm)
    print('My answer: ')
    display(br_norm_answer)
br_norm = br_norm_answer.copy()

#### Merge

In [ ]:
# Now that we have normalized broker and fund admin data, 
# we will merge the 2 dataframes

In [ ]:
# One-to-one join on unique_id
pos = (fa_norm.add_suffix("_fa")
       .merge(br_norm.add_suffix("_br"),
              left_on="security_id_fa", right_on="security_id_br",
              how="outer", validate="one_to_one"))

display(pos)

#### Compute breaks and make a breaks report

In [ ]:
# TODO: Build break table
# 1. Keep only 1 security_id column

# 2. Compute quantity breaks 'qty_diff'

# 3. Compute average cost difference 'avg_cost_diff'

# 4. Output a clean dataframe, sort values by security_id
break_pos = pd.DataFrame() # UPDATE CODE HERE

display(break_pos)

In [ ]:
if break_pos.equals(break_pos_answer):
    print('Congratulations')
else:
    print('Your answer: ')
    display(break_pos)
    print('My answer: ')
    display(break_pos_answer)
break_pos = break_pos_answer.copy()

## Step 2 — Trades + EOD P&L (Trade Date 2025‑09‑12)

**Prompt:**
1) Load **both** trade files; compute `notional`.
2) Merge marks; compute per-trade EOD `pnl`.
3) Aggregate per `unique_id` and compare Admin vs Broker (identify ES break).
4) (Optional) Add carry P&L from 09/11 to 09/12 and show total P&L.

In [ ]:
# Load trades
fa_trd = pd.read_csv(f"{TRD_DIR}/fund_admin_trades_2025-09-12.csv", parse_dates=["trade_date","settle_date"])
br_trd = pd.read_csv(f"{TRD_DIR}/broker_trades_2025-09-12.csv", parse_dates=["trade_date","settle_date"])

In [ ]:
# View the data and describe the data verbally

In [ ]:
# Display fund admin data


In [ ]:
# Display broker data


In [ ]:
# This time we have not normalized the data for you, 
# TODO: Normalize trades data
# Feel free to refer above to copy code
# The schema requirements are below

# Converters to update broker/fund admin column names to LTA schema names
trd_col_converters = {
    'date': 'date',
    'ticker': 'security_description',
    'unique_id': 'security_id',
    'quantity': 'quantity',
    'price': 'price',
    'contract_multiplier': 'multiplier',
}

# Final columns that should be in your trades dataframe
keep_trd_cols = [
    'security_description',
    'security_id',
    'trade_date',
    'settle_date',
    'price',
    'commissions',
    'quantity',
    'multiplier']

In [ ]:
# TODO: normalize broker trade data to LTA Schema Requirements, sort values by security_id
# Name the normalized dataframe br_trd_norm
br_trd_norm = pd.DataFrame() # YOUR CODE HERE

In [ ]:
if br_trd_norm.equals(br_trd_norm_answer):
    print('Congratulations')
else:
    print('Your answer: ')
    display(br_trd_norm)
    print('My answer: ')
    display(br_trd_norm_answer)
br_trd_norm = br_trd_norm_answer.copy()

In [ ]:
# TODO: normalize fund admin trade data to LTA Schema Requirements, sort values by security_id
# Name the normalized dataframe fa_trd_norm

# Hints available:
# 1. here are the columns you need to group by and aggregate over
d = fa_trd.copy()
d = d.rename(columns={'unique_id':'security_id'})
trd_grp_cols =     ['ticker', 
                    'security_id',
                    'trade_date',
                    'settle_date',
                    'contract_multiplier']


# Groupby and aggregate
# 1. Total quantity
# 2. Total commissions
# 3. Price (how will you agg this?)


# Reset Index, rename columns, keep LTA schema columns, sort_values

fa_trd_norm = pd.DataFrame()

In [ ]:
if fa_trd_norm.equals(fa_trd_norm_answer):
    print('Congratulations')
else:
    print('Your answer: ')
    display(fa_trd_norm)
    print('My answer: ')
    display(fa_trd_norm_answer)
fa_trd_norm = fa_trd_norm_answer.copy()

In [ ]:
# Now that we have our daily trades normalized
# We want to calculate our daily PNL to see if broker and fund admin are matching
# Here are the end of day marks for 09/12
MARKS = f"{DATA}/marks_2025-09-12.csv"
marks  = pd.read_csv(MARKS, parse_dates=["date"])
display(marks)

In [ ]:
# Function to merge closing marks with normalized trades
def merge_close_marks_on_trades(df, marks):
    d = df.copy()
    out = df.merge(marks[["security_id","close"]], on="security_id", how="left", validate="many_to_one")
    return out

fa_trd_cl = merge_close_marks_on_trades(fa_trd_norm, marks)

br_trd_cl = merge_close_marks_on_trades(br_trd_norm, marks)

In [ ]:
print('Fund admin trades: ')
display(fa_trd_cl)
print('Broker trades: ')
display(br_trd_cl)

In [ ]:
# TODO: Write a function that takes in a trades dataframe and calculates trade PNL
# You can ignore comms -- but tell us how you would account for it
# The returned df should have 2 additional columns called 'notional' and 'pnl'

In [ ]:
def calculate_notional_and_pnl(df):
    # YOUR CODE HERE
    return pd.DataFrame() # UPDATE THIS 

In [ ]:
fa_p = calculate_notional_and_pnl(fa_trd_cl)
assert type(fa_p) == pd.core.frame.DataFrame

In [ ]:
if fa_p.equals(fa_p_answer):
    print('Congratulations')
else:
    print('Your answer: ')
    display(fa_p)
    print('My answer: ')
    display(fa_p_answer)
fa_p = fa_p_answer.copy()

In [ ]:
br_p = calculate_notional_and_pnl(br_trd_cl)
assert type(fa_p) == pd.core.frame.DataFrame

In [ ]:
if br_p.equals(br_p_answer):
    print('Congratulations')
else:
    print('Your answer: ')
    display(br_p)
    print('My answer: ')
    display(br_p_answer)
br_p = br_p_answer.copy()

In [ ]:
# Aggregate per instrument
fa_p = fa_p.groupby("security_id", as_index=False)["pnl"].sum().rename(columns={"pnl":"pnl_admin"})
br_p = br_p.groupby("security_id", as_index=False)["pnl"].sum().rename(columns={"pnl":"pnl_broker"})

# Compute PNL breaks
pnl_cmp = (fa_p.merge(br_p, on="security_id", how="outer").fillna(0.0))
pnl_cmp["pnl_break"] = pnl_cmp["pnl_broker"] - pnl_cmp["pnl_admin"]
pnl_cmp = pnl_cmp.sort_values("security_id").reset_index(drop=True)

In [ ]:
display(pnl_cmp)

In [ ]:
if pnl_cmp.equals(pnl_cmp_answer):
    print('Congratulations')
else:
    print('Your answer: ')
    display(pnl_cmp)
    print('My answer: ')
    display(pnl_cmp_answer)
pnl_cmp = pnl_cmp_answer.copy()

## Step 3 -- Fix the break
- Call the broker, have them fix the trade file and send you a new one
- Once they send you a fixed file, validate that the PNL break has cleared

In [ ]:
br_trd_v2 = pd.read_csv(f"{TRD_DIR}/broker_trades_2025-09-12_v2.csv", parse_dates=["trade_date","settle_date"])
print("New data: ")
display(br_trd_v2)
print("Old data: ")
display(br_trd)

In [ ]:
# TODO: recompute the pnl_cmp table with the updated broker trade file
# name the new table pnl_cmp_v2
# make sure that all variables where you are using the new data end with _v2
# The final table should be named pnl_cmp_v2

In [ ]:
pnl_cmp_v2 = pd.DataFrame()

In [ ]:
assert type(pnl_cmp_v2) == pd.core.frame.DataFrame
if pnl_cmp_v2.equals(pnl_cmp_v2_answer):
    print('Congratulations')
else:
    print('Your answer: ')
    display(pnl_cmp_v2)
    print('My answer: ')
    display(pnl_cmp_v2_answer)
pnl_cmp_v2 = pnl_cmp_v2_answer.copy()

## Step 4 — OOP and Class Development

I have created a simple class using OOP, take a look at the below code for 5-10 minutes,
try and connect it to concepts we went over earlier in this exercise.

Once you're done taking a look, I'll be asking questions related to:
1. What OOP concepts do you see here?
2. What are the advantages and disadvantages 
3. Quality/Extensions -- what can be improved? Changed?

In [ ]:
from abc import ABC, abstractmethod

# =============== 1) Abstraction ===============

class RecordSource(ABC):
    """
    Abstract contract for any tabular source. 
    Subclasses must return a DataFrame with a COMMON schema via normalize().
    Target schema: ['id', 'qty'] where qty is signed.
    """

    @abstractmethod
    def get_data(self) -> pd.DataFrame:
        """Return raw DataFrame (whatever columns the source has)."""
        pass

    @abstractmethod
    def normalize(self, df: pd.DataFrame) -> pd.DataFrame:
        """Map raw to common schema ['id','qty'] (signed)."""
        pass

# =============== 2) Inheritance ===============

class AdminSource(RecordSource):
    """
    Raw schema: ['record_id','quantity','sign'] 
      - quantity is unsigned, direction in 'sign' (+1 or -1).
    """

    def __init__(self, raw_df: pd.DataFrame):
        self.raw = raw_df

    def get_data(self) -> pd.DataFrame:
        return self.raw.copy()

    def normalize(self, df: pd.DataFrame) -> pd.DataFrame:
        out = df.rename(columns={'record_id':'id', 'quantity':'quantity_abs'})
        out['qty'] = out['quantity_abs'] * out['sign']
        return out[['id','qty']]


class BrokerSource(RecordSource):
    """
    Raw schema: ['id','qty'] 
      - qty already signed
    """

    def __init__(self, raw_df: pd.DataFrame):
        self.raw = raw_df

    def get_data(self) -> pd.DataFrame:
        return self.raw.copy()

    def normalize(self, df: pd.DataFrame) -> pd.DataFrame:
        return df[['id','qty']].copy()

# =============== 3) Composition ===============

class SimpleReconciler:
    def __init__(self, left: RecordSource, right: RecordSource):
        self.left = left
        self.right = right

    def breaks(self) -> pd.DataFrame:
        l_norm = self.left.normalize(self.left.get_data()).add_suffix('_L')
        r_norm = self.right.normalize(self.right.get_data()).add_suffix('_R')

        merged = l_norm.merge(
            r_norm,
            left_on='id_L',
            right_on='id_R',
            how='outer',
            validate='one_to_one'
        )

        merged = merged.rename(columns={'id_L':'id'}).drop(columns=['id_R'])
        merged['qty_diff'] = merged['qty_R'].fillna(0) - merged['qty_L'].fillna(0)
        return merged[['id','qty_L','qty_R','qty_diff']].sort_values('id')